# Navigation

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

The environment is already saved in the Workspace and can be accessed at the file path provided below.  Please run the next code cell without making any changes.

In [1]:
from unityagents import UnityEnvironment
import numpy as np

# please do not modify the line below
env = UnityEnvironment(file_name="./Banana")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [2]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [3]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agent while it is training**, and you should set `train_mode=True` to restart the environment.

In [5]:
env_info = env.reset(train_mode=True)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 2.0


When finished, you can close the environment.

In [ ]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agent while it is training.  However, **_after training the agent_**, you can download the saved model weights to watch the agent on your own machine! 

In [6]:
from agent.dqn_agent import Agent
import unityagents
from collections import deque
from utils.utils import timed
import torch
import pickle

class AgentTrainer():
    def __init__(self,
                 env:unityagents.environment.UnityEnvironment,
                 max_t = 1000,
                 max_n_episodes = 2000
                ):
        self.env = env
        self.max_t=max_t
        self.max_n_episodes =max_n_episodes
        self.brain_name = env.brain_names[0]
    
    @timed()
    def env_step(self,action):

        env_info = self.env.step(action)[self.brain_name] 
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        
        #print('next state, reward,done',next_state,reward,done)
        return next_state,reward,done
    
    @timed()
    def train_agent(self,
                    agent:Agent):
        """Deep Q-Learning.

        Params
        ======
            agent:Agent
        """
        scores = []                        # list containing scores from each episode
        scores_window = deque(maxlen=100)  # last 100 scores
        
        last_scores_window_mean = None
        
        solved= False
        
        for i_episode in range(1, self.max_n_episodes+1):
            env_info = self.env.reset(train_mode=True)[self.brain_name]
            #print(env_info)
            state = env_info.vector_observations[0]            # get the current state

            score = 0
            for t in range(self.max_t):
                #print(t)
                action = agent.act(state) 
                #print('Epsilon',next(agent.epsilon_gen))
                next_state,reward,done = self.env_step(action)
                agent.step(state, action, reward, next_state, done)
                state = next_state
                score += reward
                if done:
                    break 
                    
            scores_window.append(score)       # save most recent score
            scores.append(score)              # save most recent score
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
            if i_episode % 100 == 0:
                scores_window_mean = np.mean(scores_window)    
                print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, scores_window_mean))
                
                
#             if i_episode% 300==0:
#                 scores_window_mean = np.mean(scores_window)
                
#                 if (last_scores_window_mean and 
#                     abs(scores_window_mean-last_scores_window_mean <0.1)):
#                         print(f'Difference between two trainings ({last_scores_window_mean}, {scores_window_mean}) between the last 2 windows of size 300 too low. Stopping training.')
#                         break
                        
#                 last_scores_window_mean = scores_window_mean
                        
            if np.mean(scores_window)>=13.0 and not solved:
                solved= True
                print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
                
        #self.env.close()
        self.scores = scores
        torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
        return scores
    
    def save_scores(self,hyperparams,filename):
        obj = {'scores':self.scores,
               'hyperparams':hyperparams}
        
        with open(filename,'wb') as f:
            pickle.dump(obj,f)

In [7]:
import os
os.environ['profiling'] = "false"

In [8]:
from agent.ddqn_agent import Agent as DDQN_Agent
from agent.dqn_agent import Agent as DQN_Agent
from agent.ddqn_prioritized_replay_agent import Agent as PR_DDQN_Agent
agent_configs = [
#                   ({'topology':[128,128,64,64,32,32,16,16,8,8],
#                      'eps_start':1.0, 
#                      'eps_end':0.01,
#                      'eps_decay':0.9999,
#                      'description':'DQN_deep_topology'},DQN_Agent),
#                  ({'topology':[256, 128, 32],
#                      'eps_start':1.0, 
#                      'eps_end':0.01,
#                      'eps_decay':0.9999,
#                      'description':'DQN_wide_topology'},DQN_Agent),
#                  ({'topology':[128,128,64,64,32,32,16,16,8,8],
#                      'eps_start':1.0, 
#                      'eps_end':0.01,
#                      'eps_decay':0.9999,
#                      'description':'DDQN_deep_topology'},DDQN_Agent),
#                  ({'topology':[256, 128, 32],
#                      'eps_start':1.0, 
#                      'eps_end':0.01,
#                      'eps_decay':0.9999,
#                      'description':'DDQN_wide_topology_2'},DDQN_Agent),
                 ({'topology':[256, 128, 32],
                     'eps_start':1.0, 
                     'eps_end':0.01,
                     'eps_decay':0.9999,
                     'per_alpha':0,
                     'description':'Prioritized_Replay_DDQN_wide_topology'},PR_DDQN_Agent)
                ]


In [9]:
for hyperparams, Agent in agent_configs:
    print('\n\nRunning',hyperparams)
    print('Agent type',Agent)
    agent = Agent(state_size, 
                  action_size,
                  hyperparams)
    
    trainer = AgentTrainer(env,
                       max_t=1000,
                       max_n_episodes=2000)
    
    trainer.train_agent(agent)
    trainer.save_scores(hyperparams,
                        hyperparams['description'])



Running {'topology': [256, 128, 32], 'eps_start': 1.0, 'eps_end': 0.01, 'eps_decay': 0.9999, 'per_alpha': 0, 'description': 'Prioritized_Replay_DDQN_wide_topology'}
Agent type <class 'agent.ddqn_prioritized_replay_agent.Agent'>
Episode 35	Average Score: 0.66

KeyboardInterrupt: 

In [ ]:
np.seterr(all='raise')

In [ ]:
import importlib
import utils
import agent

importlib.reload(utils.utils)
importlib.reload(agent.ddqn_prioritized_replay_agent)


In [ ]:
import pickle
import pandas as pd
%matplotlib inline

with open('DDQN_wide_topology','rb') as f:
    data = pickle.load(f)

df = pd.DataFrame({'data':data['scores']})
df['rolling_mean'] = df['data'].rolling(50).mean()
df.plot()

In [ ]:
import pickle
import pandas as pd
%matplotlib inline

with open('DQN_wide_topology','rb') as f:
    data = pickle.load(f)

df = pd.DataFrame({'data':data['scores']})
df['rolling_mean'] = df['data'].rolling(50).mean()
df.plot()

In [ ]:
import pickle
import pandas as pd
%matplotlib inline

with open('DQN_deep_topology','rb') as f:
    data = pickle.load(f)

df = pd.DataFrame({'data':data['scores']})
df['rolling_mean'] = df['data'].rolling(50).mean()
df.plot()

In [ ]:
import pickle
import pandas as pd
%matplotlib inline

with open('DDQN_deep_topology','rb') as f:
    data = pickle.load(f)

df = pd.DataFrame({'data':data['scores']})
df['rolling_mean'] = df['data'].rolling(50).mean()
df.plot()

In [ ]:
def epsilon_generator(eps_start:float,
                      eps_end:float,
                      eps_decay:float):
    eps = eps_start
    while True:
        yield eps
        eps = max(eps_end, eps_decay*eps)

gen = epsilon_generator(eps_start=1.0, 
                        eps_end=0.01,
                        eps_decay=0.9999,)

epsilons = [next(gen) for i in range(300*300)]
pd.DataFrame({'epsilons':epsilons}).plot()


In [ ]:
import torch
# Save
torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')

In [ ]:
import torch
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

# Test

In [ ]:
scores = []
for i in range(100):
    env_info = env.reset(train_mode=True)[brain_name] # reset the environment
    state = env_info.vector_observations[0]            # get the current state
    score = 0                                          # initialize the score
    while True:
        action = agent.act(state,training=False)     # select an action
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break

    print("Score: {}".format(score))
    scores.append(score)

In [ ]:
#env.close()
agent

In [ ]:
from utils.utils import times
for key,val in times.items():
    print(key,np.mean(val)*1000)

In [ ]:
import time
@timed()
def f():
    time.sleep(1)
    return 1

In [ ]:
f()

In [ ]:
import numpy as np


t= SumTree(10)
t.add(1,1)
t.add(1,1)
len(t)      

In [ ]:
from collections import namedtuple
import random
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

            

In [ ]:
buffer = PrioritizedReplayBuffer(buffer_size=100, batch_size=16, seed=1)
buffer.add(state=1,
           action=1,
           reward=1,
           next_state=2,
           done =False)

In [ ]:
buffer.add(state=2,
           action=2,
           reward=2,
           next_state=3,
           done =False)

In [ ]:
indices,_=buffer.sample()

In [ ]:
buffer.update_batches(indices,indices)

In [ ]:
buffer.tree.tree

In [ ]:
from torch import nn
import torch
loss = nn.MSELoss(reduce=False)
input = torch.randn(5, 1)
target = torch.randn(5, 1)
torch.Tensor([1,2])

In [ ]:
input

In [ ]:
target

In [ ]:
loss(input,target).mean()

In [ ]:
import numpy as np
(np.square(target-input)).sum()/5

In [ ]:
weights = np.array([0.1,0.2,0.3,0.4,0.5])

In [ ]:
(weights*np.square(target-input).reshape(-1)).mean()

In [ ]:
i_w = input.reshape(-1)
t_w = target.reshape(-1)

(loss(i_w,t_w)*torch.from_numpy(weights).float()).mean()

In [ ]:
loss(i_w,t_w)*0.2

In [ ]:
input.shape

In [ ]:
input.reshape(-1)*torch.from_numpy(np.square(weights)).float()

In [ ]:
torch.from_numpy(np.square(weights).reshape(-1,1)).shape

In [ ]:
input.reshape(-1).shape